In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input, Flatten, Conv1D
from tensorflow.keras.utils import to_categorical
import numpy as np
import os
import pandas as pd
from dtaidistance import dtw

In [2]:
def euclideanDistance(x, y):
    dist = tf.sqrt(float(tf.reduce_sum(tf.square(x - y))))
    return dist

def DTW_TF(S, S1, d=euclideanDistance):
    cost_matrix = []
    cost_matrix.append([0, *([1234567891011] * S1.shape[0])])
    for i in range(1,S.shape[0]+1):
        sub_cost_j = [1234567891011]
        for j in range(1, S1.shape[0]+1):
            dst = d(S[i-1], S1[j-1])
            mat_dt = tf.stack([
            dst + sub_cost_j[j-1],
            dst + cost_matrix[i-1][j-1],
            dst + cost_matrix[i-1][j]
            ])
            sub_cost_j.append(tf.reduce_min(mat_dt))
        cost_matrix.append(tf.stack(sub_cost_j))
    return DTW_minimal_path(tf.stack(cost_matrix))

def DTW_minimal_path(cost_mat):
    i = tf.constant(cost_mat.shape[0]) - 1
    j = tf.constant(cost_mat.shape[1]) - 1
    cost = cost_mat[i, j]
    compteur = 0
    path_input = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)
    path_input = path_input.write(compteur, i-1)

    path_output = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)
    path_output = path_output.write(compteur, j-1)
    while tf.greater(i, 0) and tf.greater(j, 0):
        compteur += 1 
        cost_min = tf.stack([
            cost_mat[i-1, j-1],
            cost_mat[i, j-1],
            cost_mat[i-1, j]
        ])
        n_min = tf.math.argmin(cost_min)
        if tf.equal(n_min,0):
            path_input = path_input.write(compteur, i-2)
            path_output = path_output.write(compteur, j-2)
            i += -1
            j += -1
        elif tf.equal(n_min,1):
            path_input = path_input.write(compteur, i-1)
            path_output = path_output.write(compteur, j-2)
            j += -1
        elif tf.equal(n_min, 2):
            path_input = path_input.write(compteur, i-2)
            path_output = path_output.write(compteur, j-1)
            i += -1
    return path_input.stack()[:-1][::-1], path_output.stack()[:-1][::-1], cost

        

In [3]:
S = [1, 3, 3, 3,2, 0, 0, 1]
S1 = [0, 1, 3, 2, 2, 0, 1]

# distance = dtw.distance(S, S1)
# print(distance)
# print(dtw.warping_paths(S, S1)[1])

S = tf.convert_to_tensor(S)
S1 = tf.convert_to_tensor(S1)

i = 7
j = 7
DTW_TF(S, S1)


(<tf.Tensor: shape=(10,), dtype=int32, numpy=array([0, 0, 1, 2, 3, 4, 4, 5, 6, 7])>,
 <tf.Tensor: shape=(10,), dtype=int32, numpy=array([0, 1, 2, 2, 2, 3, 4, 5, 5, 6])>,
 <tf.Tensor: shape=(), dtype=float32, numpy=1.0>)

In [4]:
@tf.function
def convolution_1D(inputs, weights):
    weights = tf.linalg.matrix_transpose(weights)
    final_shape = (inputs.shape[-2] - weights.shape[-1] + 1, weights.shape[0])
    output_list = []
        #output = tf.reshape(tf.linalg.trace(tf.linalg.matmul(weights, inputs[0,0:0+weights.shape[1]])), (1,*final_shape[2:]))
    for j in range(0, inputs.shape[-2] - weights.shape[-1] + 1):
        output_list.append(tf.linalg.trace(tf.linalg.matmul(inputs[j:j+weights.shape[-1]], weights)))
    output_final = tf.stack(output_list)
    return output_final

def stride_alignment(strid_input, weights):
        output_list = []
        for filt in range(weights.shape[0]):
            t_input = strid_input
            t_weight = weights[filt]
            path_input, path_weight, _ = DTW_TF(t_input, t_weight)
            weights_align = tf.gather(t_weight, indices=path_weight)
            inputs_n = tf.gather(t_input, indices=path_input)
            weights_align = tf.linalg.matrix_transpose(weights_align)
            output_list.append(tf.linalg.trace(tf.linalg.matmul(inputs_n, weights_align)))
        print(tf.stack(output_list))
        return tf.stack(output_list)

@tf.function
def conv1D_weight_alignment(inputs, weights):
    final_shape = (inputs.shape[-2] - weights.shape[-2] + 1, weights.shape[0])
    output_final = []
    tensor_iter =  tf.constant([*range(0, inputs.shape[-2] - weights.shape[-2] + 1)])
    output_final = tf.map_fn(lambda j: stride_alignment(tf.slice(inputs, (j, 0) , (weights.shape[-2:])), weights), tensor_iter, fn_output_signature=tf.float32)
    print(output_final)
    return output_final

class CNN1D(keras.layers.Layer):
    def __init__(self, n_filters=8, kernel_size=3):
        super(CNN1D, self).__init__()
        self.n_filters = n_filters
        self.kernel_size = kernel_size
        self.b = self.add_weight(shape=(n_filters,), initializer="zeros", trainable=True)
    
    def build(self, input_shape): 
        self.w = self.add_weight(
            shape=(self.n_filters, self.kernel_size, int(input_shape[-1])),
            initializer="glorot_normal", trainable=True
        )
    def call(self, inputs):
        output = tf.map_fn(lambda inp: convolution_1D(inp, self.w) + self.b, inputs)
        return tf.nn.relu(output)
    
class DWA_CNN(CNN1D):
    def call(self, inputs):
        output = tf.map_fn(lambda inp: conv1D_weight_alignment(inp, self.w) + self.b, inputs)
        return tf.nn.relu(output)

In [5]:
randi = np.random.random((100,12, 3))
y_train = np.random.randint(1,3, 100)
y_train = to_categorical(y_train)

In [6]:
tf.stack([tf.constant(0)])

<tf.Tensor: shape=(1,), dtype=int32, numpy=array([0])>

In [7]:
tf.random.set_seed(1234)
model = Sequential([
    Input(randi.shape[1:]),
    DWA_CNN(5, 3),
    Flatten(),
    Dense(3, activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy', metrics='accuracy')
model.fit(randi, y_train, epochs=20)

Tensor("map/while/stack_65:0", shape=(5,), dtype=float32)
Tensor("map/TensorArrayV2Stack/TensorListStack:0", shape=(10, 5), dtype=float32)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dwa_cnn (DWA_CNN)            (None, 10, 5)             50        
_________________________________________________________________
flatten (Flatten)            (None, 50)                0         
_________________________________________________________________
dense (Dense)                (None, 3)                 153       
Total params: 203
Trainable params: 203
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
4/4 [==============================] - 2s 587ms/step - loss: 0.9394 - accuracy: 0.5000
Epoch 2/20
4/4 [==============================] - 3s 647ms/step - loss: 0.9014 - accuracy: 0.5000
Epoch 3/20
4/4 [==============================] - 3s 

In [119]:
print(randi.shape[1:])
tf.random.set_seed(1234)
model_conv = Sequential([
    Input(randi.shape[1:]),
    CNN1D(5,1),
    Flatten(),
    Dense(3, activation="softmax")   
])

model_conv.summary()
model_conv.compile(loss='categorical_crossentropy', metrics='accuracy')
model_conv.fit(randi, y_train, epochs=20)

(12, 3)
Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cn_n1d_4 (CNN1D)             (None, 12, 5)             20        
_________________________________________________________________
flatten_8 (Flatten)          (None, 60)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 3)                 183       
Total params: 203
Trainable params: 203
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
4/4 [==============================] - 0s 8ms/step - loss: 1.2446 - accuracy: 0.1400
Epoch 2/20
4/4 [==============================] - 0s 28ms/step - loss: 1.1677 - accuracy: 0.2600
Epoch 3/20
4/4 [==============================] - 0s 18ms/step - loss: 1.1220 - accuracy: 0.2900
Epoch 4/20
4/4 [==============================] - 0s 20ms/step - loss: 1.0904 - accuracy

In [8]:
from sklearn.preprocessing import MinMaxScaler
from random import shuffle

def get_filepaths(mainfolder):
    """
    Searches a folder for all unique files and compile a dictionary of their paths.
    Parameters
    --------------
    mainfolder: the filepath for the folder containing the data
    Returns
    --------------
    training_filepaths: file paths to be used for training
    testing_filepaths:  file paths to be used for testing
    """
    training_filepaths = {}
    testing_filepaths  = {}
    folders = os.listdir(mainfolder)
    for folder in folders:
        fpath = mainfolder + "/" + folder
        if os.path.isdir(fpath) and "MODEL" in folder:
            filenames = os.listdir(fpath)
            for filename in filenames[:int(round(0.8*len(filenames)))]:
                fullpath = fpath + "/" + filename
                training_filepaths[fullpath] = folder
            for filename1 in filenames[int(round(0.8*len(filenames))):]:
                fullpath1 = fpath + "/" + filename1
                testing_filepaths[fullpath1] = folder
    return training_filepaths, testing_filepaths

def get_labels(mainfolder):
    """ Creates a dictionary of labels for each unique type of motion """
    labels = {}
    label = 0
    for folder in os.listdir(mainfolder):
        fpath = mainfolder + "/" + folder
        if os.path.isdir(fpath) and "MODEL" in folder:
            labels[folder] = label
            label += 1
    return labels

def get_data(fp, labels, folders, norm, std, center):
    """
    Creates a dataframe for the data in the filepath and creates a one-hot
    encoding of the file's label
    """
    data = pd.read_csv(filepath_or_buffer=fp, sep=' ', names = ["X", "Y", "Z"])
    if norm and not std:
        normed_data = norm_data(data)
    elif std and not norm:
        stdized_data = std_data(data)
    elif center and not norm and not std:
        cent_data = subtract_mean(data)

    one_hot = np.zeros(7)
    file_dir = folders[fp]
    label = labels[file_dir]
    one_hot[label] = 1
    return normed_data, one_hot, label

# Normalizes the data by removing the mean

def subtract_mean(input_data):
    # Subtract the mean along each column
    centered_data = input_data - input_data.mean()
    return centered_data


def norm_data(data):
    """
    Normalizes the data.
    For normalizing each entry, y = (x - min)/(max - min)
    """
    c_data = subtract_mean(data)
    mms = MinMaxScaler()
    mms.fit(c_data)
    n_data = mms.transform(c_data)
    return n_data

def standardize(data):
    c_data = subtract_mean(data)
    std_data = c_data/ pd.std(c_data)
    return std_data

def vectorize(normed):
    """
    Uses a sliding window to create a list of (randomly-ordered) 300-timestep
    sublists for each feature.
    """
    sequences = [normed[i:i+150] for i in range(len(normed)-150)]
    shuffle(sequences)
    sequences = np.array(sequences)
    return sequences

def build_inputs(files_list, accel_labels, file_label_dict, norm_bool, std_bool, center_bool):
    X_seq    = []
    y_seq    = []
    labels = []
    for path in files_list:
        normed_data, target, target_label = get_data(path, accel_labels, file_label_dict, norm_bool, std_bool, center_bool)
        input_list = vectorize(normed_data)
        for inputs in range(len(input_list)):
            X_seq.append(input_list[inputs])
            y_seq.append(list(target))
            labels.append(target_label)
    X_ = np.array(X_seq)
    y_ = np.array(y_seq)
    return X_, y_, labels

mainpath = "../data/HMP_Dataset"


activity_labels                  = get_labels(mainpath)
training_dict, testing_dict      = get_filepaths(mainpath)
training_files                   = list(training_dict.keys())
testing_files                   = list(testing_dict.keys())

    # build training inputs and labels
X_train, y_train, train_labels = build_inputs(
    training_files,
    activity_labels,
    training_dict,
    True, False, False)


X_test, y_test, test_labels = build_inputs(
    testing_files,
    activity_labels,
    testing_dict,
    True, False, False)

shuffle = np.random.permutation(len(X_train))
X_train = X_train[shuffle]
y_train = y_train[shuffle]

In [85]:
print(X_train.shape, X_test.shape)

(9408, 150, 3) (2352, 150, 3)


In [ ]:
tf.random.set_seed(1234)
model = Sequential([
    Input(X_train.shape[1:]),
    CNN1D(10, 4),
    Flatten(),
    Dense(7, activation='softmax')   
])

model.summary()
model.compile(loss='categorical_crossentropy', metrics='accuracy')
model.fit(X_train, y_train, epochs=10, validation_data = (X_test, y_test))

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cn_n1d_7 (CNN1D)             (None, 147, 10)           130       
_________________________________________________________________
flatten_11 (Flatten)         (None, 1470)              0         
_________________________________________________________________
dense_11 (Dense)             (None, 7)                 10297     
Total params: 10,427
Trainable params: 10,427
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
233/294 [======================>.......] - ETA: 9s - loss: 0.7428 - accuracy: 0.7504

In [86]:
tf.random.set_seed(1234)
model = Sequential([
    Input(X_train.shape[1:]),
    Conv1D(10,3, activation='relu'),
    Flatten(),
    Dense(7, activation='softmax')   
])

model.summary()
model.compile(loss='categorical_crossentropy', metrics='accuracy')
model.fit(X_train, y_train, epochs=10, validation_data = (X_test, y_test))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 148, 10)           100       
_________________________________________________________________
flatten_1 (Flatten)          (None, 1480)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 10367     
Total params: 10,467
Trainable params: 10,467
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
294/294 [==============================] - 1s 4ms/step - loss: 0.7065 - accuracy: 0.7518 - val_loss: 0.4765 - val_accuracy: 0.8486
Epoch 2/10
294/294 [==============================] - 1s 3ms/step - loss: 0.3375 - accuracy: 0.8846 - val_loss: 0.3426 - val_accuracy: 0.8856
Epoch 3/10
294/294 [==============================] - 1s 3ms/step - loss: 0.2520 - accuracy: 0.9

In [ ]:
tf.random.set_seed(1234)
model = Sequential([
    Input(X_train.shape[1:]),
    DWA_CNN(10, 3),
    Flatten(),
    Dense(7, activation='softmax')   
])

model.summary()
model.compile(loss='categorical_crossentropy', metrics='accuracy')
model.fit(X_train, y_train, epochs=10, validation_data = (X_test, y_test))

Tensor("map/while/stack_130:0", shape=(10,), dtype=float32)
Tensor("map/TensorArrayV2Stack/TensorListStack:0", shape=(148, 10), dtype=float32)
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dwa_cnn_1 (DWA_CNN)          (None, 148, 10)           100       
_________________________________________________________________
flatten_1 (Flatten)          (None, 1480)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 10367     
Total params: 10,467
Trainable params: 10,467
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
294/294 [==============================] - 14417s 49s/step - loss: 0.6788 - accuracy: 0.7706 - val_loss: 0.5398 - val_accuracy: 0.8384
Epoch 2/10
 77/294 [======>.......................] - ETA: 1:32:00 - loss: 0.3889 - accurac

In [117]:
weights = tf.constant([1,3,4,5], shape=(1,2,2))
print(weights)
weights.shape[:0:-1]

tf.Tensor(
[[[1 3]
  [4 5]]], shape=(1, 2, 2), dtype=int32)


TensorShape([2, 2])